In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv("../../data/total_4am.csv", index_col=0)
df

FileNotFoundError: [Errno 2] No such file or directory: 'data/total_4am.csv'

In [ ]:
import pandas as pd
import numpy as np

# Assuming the dataframe is named df
# Calculate the price change
df['price_change'] = df['close'].diff()

# Calculate the gains and losses
df['gain'] = np.where(df['price_change'] > 0, df['price_change'], 0)
df['loss'] = np.where(df['price_change'] < 0, -df['price_change'], 0)

# Calculate the average gain and average loss
window_length = 14
df['avg_gain'] = df['gain'].rolling(window=window_length, min_periods=1).mean()
df['avg_loss'] = df['loss'].rolling(window=window_length, min_periods=1).mean()

# Calculate the RS (Relative Strength) and RSI
df['rs'] = df['avg_gain'] / df['avg_loss']
df['rsi'] = 100 - (100 / (1 + df['rs']))

# Implement RSI strategy for long positions only
df['signal'] = 0  # Default to no position
df['signal'] = np.where(df['rsi'] < 30, 1, df['signal'])   # Buy signal when RSI < 30
df['signal'] = np.where(df['rsi'] > 70, -1, df['signal'])  # Exit signal when RSI > 70

# Manage positions
df['position'] = 0
holding_position = False

for i in range(1, len(df)):
    if df['signal'].iloc[i] == 1 and not holding_position:
        df['position'].iloc[i] = 1
        holding_position = True
    elif df['signal'].iloc[i] == -1 and holding_position:
        df['position'].iloc[i] = 0
        holding_position = False
    else:
        df['position'].iloc[i] = df['position'].iloc[i-1]

# Calculate the strategy returns (only when in a long position)
df['strategy_returns'] = df['position'].shift(1) * df['close'].pct_change()

# Calculate the cumulative returns
df['cumulative_returns'] = (1 + df['strategy_returns']).cumprod()

# Calculate the benchmark cumulative returns (buy and hold strategy)
df['benchmark_returns'] = (1 + df['close'].pct_change()).cumprod()

# Display the results
print(df[['day_starting_at_4am', 'close', 'rsi', 'signal', 'position', 'cumulative_returns', 'benchmark_returns']].tail(10))

df.to_csv("../../results/rsi1.csv")
